In [1]:
from vnstock import *
import yfinance as yf
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [5]:
#MODEL
list_i = []
list_pcapercent = []
list_return_price = []
list_rmse, list_mse, list_mae =[], [], []
list_k = []
list_r2_train = []
list_r2_test = []

scaler = StandardScaler()
pca = PCA(n_components = 20)

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    a = data.loc[:, ~data.columns.isin(['Today'])]

    a_scale = scaler.fit_transform(a)
    a_scale_df = pd.DataFrame(a_scale, columns=a.columns)
    pF = pca.fit_transform(a_scale_df)
    pF_df = pd.DataFrame(data = pF) 
    ss = pca.explained_variance_ratio_
    total = sum(ss)
    list_pcapercent.append(total)
    b = pd.DataFrame()
    b['Today'] = data['Today'].reset_index(drop=True)
    pF_final = pd.concat([pF_df, b], axis=1)
    
    X = pF_final.loc[:, ~pF_final.columns.isin(['Today'])]
    y = pF_final['Today']

    list_k1=[]
    list_score=[]
    for k in np.arange(0.1,0.5,0.01):
      model = Lasso(alpha=1.0)
      X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = k, random_state = 12, shuffle=False)
      model.fit(X_train, y_train)
      yhat_test = model.predict(X_test)
      score = model.score(X_test, y_test)
      list_k1.append(k)
      list_score.append(score)

    score_max = max(list_score)
    a = list_score.index(score_max)
    k = list_k1[a]

    model = Lasso(alpha=1.0)
    X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
    model.fit(X_train,y_train)
    yhat_test = model.predict(X_test)
    r2_train = model.score(X_train, y_train)
    r2_test = model.score(X_test, y_test)
    mse = mean_squared_error(y_test, yhat_test, squared=True)
    rmse = mean_squared_error(y_test, yhat_test, squared=False)
    mae = mean_absolute_error(y_test, yhat_test)

    X_predict = [X.iloc[-1].values.tolist()]
    Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i, f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f'MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',f'pca: {total:.2f}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -0.05  R2_train: 0.12  R2_test: 0.19  RMSE: 1.73 MSE: 2.98 MAE: 1.19  test size: 0.14 pca: 0.91
Stock: BCM  Predict: 0.17  R2_train: 0.05  R2_test: 0.05  RMSE: 1.83 MSE: 3.34 MAE: 1.22  test size: 0.42 pca: 0.90
Stock: VHM  Predict: 0.03  R2_train: 0.00  R2_test: -0.00  RMSE: 2.19 MSE: 4.78 MAE: 1.61  test size: 0.21 pca: 0.90
Stock: VIC  Predict: 0.01  R2_train: 0.00  R2_test: -0.00  RMSE: 2.13 MSE: 4.53 MAE: 1.36  test size: 0.47 pca: 0.90
Stock: VRE  Predict: 0.12  R2_train: 0.01  R2_test: 0.01  RMSE: 1.72 MSE: 2.96 MAE: 1.28  test size: 0.18 pca: 0.91
Stock: BVH  Predict: 0.16  R2_train: 0.02  R2_test: 0.01  RMSE: 2.15 MSE: 4.63 MAE: 1.48  test size: 0.49 pca: 0.90
Stock: POW  Predict: -0.02  R2_train: 0.04  R2_test: 0.07  RMSE: 1.59 MSE: 2.52 MAE: 1.17  test size: 0.21 pca: 0.90
Stock: GAS  Predict: 0.15  R2_train: 0.02  R2_test: 0.02  RMSE: 1.58 MSE: 2.50 MAE: 1.17  test size: 0.37 pca: 0.90
Stock: ACB  Predict: 0.11  R2_train: 0.00  R2_test: -0.00  RMSE: 1.6

In [6]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k, 'pca': list_pcapercent})
KQ.to_excel("Kết quả sau PCA.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE,test size,pca
0,SSI,[-0.05075233294033904],0.116832,0.192895,2.980815,1.726504,1.189467,0.14,0.908270
1,BCM,[0.16800201736482986],0.047233,0.046445,3.340439,1.827687,1.224146,0.42,0.900623
2,VHM,[0.03156762390784285],0.000000,-0.000022,4.780981,2.186545,1.612250,0.21,0.904386
3,VIC,[0.006279038558792031],0.000000,-0.000006,4.534198,2.129366,1.356810,0.47,0.902092
4,VRE,[0.12377377481694143],0.006149,0.005774,2.963564,1.721500,1.280566,0.18,0.905858
...,...,...,...,...,...,...,...,...,...
389,TDP,[0.6678905617447236],0.054304,0.071972,6.482871,2.546148,1.725018,0.49,0.901230
390,NO1,[-0.03068694097881499],0.000000,-0.000359,11.504671,3.391854,2.560219,0.28,0.909929
391,L10,[1.8029837569128058],0.216444,0.255817,22.168232,4.708315,4.092602,0.14,0.894500
392,REE,[0.2676252459405785],0.046720,0.060479,1.708011,1.306909,0.974723,0.20,0.900736
